### Run in collab
<a href="https://colab.research.google.com/github/racousin/data_science_practice/blob/master/website/public/modules/module13/exercise/module13_exercise4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install swig==4.2.1
!pip install gymnasium==0.29.1
!pip install gymnasium[box2d]  # Install Box2D dependency for LunarLander-v3

In [18]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

# module13_exercise4 : ML - Arena <a href="https://ml-arena.com/viewcompetition/1" target="_blank"> LunarLander</a>

### Objective
Get at list an agent running on ML-Arena <a href="https://ml-arena.com/viewcompetition/1" target="_blank"> LunarLander</a> with mean reward upper than 50


You should submit an agent file named `agent.py` with a class `Agent` that includes at least the following attributes:

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import gymnasium as gym  # nécessite gymnasium[box2d] pour LunarLander-v2

# Définition du réseau de neurones pour approximer Q(s,a)
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        # Réseau fully-connected avec 2 couches cachées de 128 neurones
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc_out = nn.Linear(128, action_dim)
        # Initialisation optionnelle des poids peut être ajoutée ici si désiré

    def forward(self, state):
        # Passe avant : ReLU sur couches cachées
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc_out(x)  # sorties Q-values (une par action)

# Buffer d'expérience pour stocker et échantillonner des transitions
class ReplayBuffer:
    def __init__(self, capacity, state_dim):
        self.capacity = capacity
        self.memory = []        # liste de transitions
        self.position = 0       # index courant pour écraser les anciennes expériences
        self.state_dim = state_dim

    def add(self, state, action, reward, next_state, done):
        # Si la mémoire n'est pas encore pleine, on ajoute une nouvelle entrée
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        # Stocker la transition (on copie les tableaux pour éviter les références)
        self.memory[self.position] = (
            np.array(state, copy=True),
            action,
            reward,
            np.array(next_state, copy=True),
            done
        )
        # Incrément circulaire de la position
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        # Tirer aléatoirement batch_size transitions
        indices = np.random.choice(len(self.memory), batch_size, replace=False)
        states, actions, rewards, next_states, dones = zip(*(self.memory[i] for i in indices))
        # Convertir en tenseurs PyTorch
        states      = torch.tensor(np.array(states), dtype=torch.float32)
        actions     = torch.tensor(actions, dtype=torch.int64).unsqueeze(1)    # actions indices
        rewards     = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)  # récompenses
        next_states = torch.tensor(np.array(next_states), dtype=torch.float32)
        dones       = torch.tensor(dones, dtype=torch.float32).unsqueeze(1)    # indicateurs de fin (0.0 ou 1.0)
        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.memory)

# Agent DQN avec réseau local et réseau cible
class DQNAgent:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        # Initialiser les deux réseaux (policy et target) et l'optimiseur
        self.q_network = QNetwork(state_dim, action_dim)
        self.target_network = QNetwork(state_dim, action_dim)
        self.target_network.load_state_dict(self.q_network.state_dict())  # initialisation identique
        self.target_network.eval()  # le réseau cible n'est pas entraîné par gradient
        self.optimizer = torch.optim.Adam(self.q_network.parameters(), lr=5e-4)
        # Initialiser la mémoire d'expérience
        self.memory = ReplayBuffer(capacity=100000, state_dim=state_dim)
        # Compteur de pas pour gestion des mises à jour
        self.learn_step_counter = 0

    def select_action(self, state, epsilon):
        """Renvoie une action selon une politique epsilon-greedy."""
        if np.random.rand() < epsilon:
            # Exploration aléatoire
            return np.random.randint(self.action_dim)
        else:
            # Exploitation (on choisit l'action de Q maximale)
            state_t = torch.tensor(state, dtype=torch.float32).unsqueeze(0)  # shape (1, state_dim)
            self.q_network.eval()  # mode évaluation
            with torch.no_grad():
                q_values = self.q_network(state_t)
            self.q_network.train()  # repasse en mode entraînement
            action = int(torch.argmax(q_values, dim=1).item())
            return action

    def train_step(self, batch_size=64, gamma=0.99, tau=1e-3):
        """Effectue un pas d'apprentissage du réseau (une mise à jour de Q-network)."""
        if len(self.memory) < batch_size:
            return  # ne pas entraîner tant qu'on n'a pas assez d'échantillons
        # Échantillonner un mini-batch de transitions
        states, actions, rewards, next_states, dones = self.memory.sample(batch_size)
        # Calcul des Q-cibles avec le réseau cible (on ne calcule pas de gradients ici)
        with torch.no_grad():
            # Valeur Q max du prochain état selon le réseau cible
            q_next = self.target_network(next_states).max(dim=1, keepdim=True)[0]
            # Cible de Q: r + gamma * max(Q_next) * (1 - done)
            q_target = rewards + gamma * q_next * (1 - dones)
        # Valeur Q courante prédite par le réseau principal pour les (state, action) du batch
        q_current = self.q_network(states).gather(1, actions)  # Q(s,a) pour chaque transition du batch
        # Calcul de la perte (erreur quadratique)
        loss = F.mse_loss(q_current, q_target)
        # Rétropropagation de la perte et mise à jour des poids du Q-network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        # Mise à jour douce du réseau cible vers le Q-network (tau)
        for target_param, local_param in zip(self.target_network.parameters(), self.q_network.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

# Environnement LunarLander-v2
env = gym.make("LunarLander-v3")
state_dim = env.observation_space.shape[0]   # dimension d'état (8)
action_dim = env.action_space.n             # nombre d'actions (4)
agent = DQNAgent(state_dim, action_dim)

# Paramètres d'entraînement
num_episodes = 1000         # nombre maximal d'épisodes
max_steps = 1000            # pas max par épisode (pour éviter des boucles infinies)
target_score = 200          # score cible à atteindre en moyenne
print_interval = 10         # intervalle pour affichage des progrès
best_avg_reward = -float("inf")
best_model_path = "best_model.pth"

# Variables pour suivi de la performance
scores = []                 # liste des scores par épisode
scores_window = []          # fenêtre glissante des derniers 100 scores

# Boucle principale d'entraînement
epsilon = 1.0               # valeur initiale de epsilon (politique epsilon-greedy)
epsilon_decay = 0.995       # facteur de décroissance exponentielle de epsilon
epsilon_min = 0.01          # epsilon minimum
for episode in range(1, num_episodes + 1):
    state, _ = env.reset()  # réinitialiser l'environnement
    episode_reward = 0
    for t in range(max_steps):
        # Sélectionner une action selon la politique epsilon-greedy
        action = agent.select_action(state, epsilon)
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        episode_reward += reward
        # Ajouter la transition dans la mémoire
        agent.memory.add(state, action, reward, next_state, done)
        # Mettre à jour l'état courant
        state = next_state
        # Entraîner le réseau (toutes les 4 étapes)
        if t % 4 == 0:
            agent.train_step(batch_size=64, gamma=0.99, tau=1e-3)
        # Sortir si fin d'épisode
        if done:
            break
    # Mettre à jour epsilon (décroissance exponentielle par épisode)
    epsilon = max(epsilon * epsilon_decay, epsilon_min)
    # Enregistrer le score de l'épisode
    scores.append(episode_reward)
    scores_window.append(episode_reward)
    if len(scores_window) > 100:
        # garder une fenêtre glissante de 100 derniers épisodes
        scores_window.pop(0)
    # Calculer la récompense moyenne des 100 derniers épisodes
    avg_reward_100 = np.mean(scores_window)
    # Sauvegarder le modèle si c'est le meilleur jusqu'à présent
    if avg_reward_100 > best_avg_reward:
        best_avg_reward = avg_reward_100
        torch.save(agent.q_network.state_dict(), best_model_path)
    # Affichage périodique des statistiques d'entraînement
    if episode % print_interval == 0:
        print(f"Épisode {episode}/{num_episodes} - Score moyen (100 derniers): {avg_reward_100:.1f} - eps={epsilon:.3f}")
    # Arrêt anticipé si la moyenne sur 100 épisodes atteint la cible
    if avg_reward_100 >= target_score and episode >= 100:
        print(f"Environnement résolu en {episode} épisodes 🎉  (score moyen sur 100 eps = {avg_reward_100:.1f})")
        break

# Fin de l'entraînement
print("Meilleure moyenne obtenue sur 100 épisodes:", best_avg_reward)
# Charger le meilleur modèle sauvegardé
best_model = QNetwork(state_dim, action_dim)
best_model.load_state_dict(torch.load(best_model_path))
best_model.eval()

# Évaluation du modèle entraîné sur 100 épisodes pour vérifier la performance > 200
eval_episodes = 1000
eval_rewards = []
for i in range(eval_episodes):
    state, _ = env.reset()
    episode_sum = 0
    while True:
        # Sélectionner action de façon déterministe (epsilon=0, politique purement optimisée)
        state_t = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            q_vals = best_model(state_t)
        action = int(torch.argmax(q_vals, dim=1).item())
        # Agir dans l'environnement
        next_state, reward, terminated, truncated, info = env.step(action)
        episode_sum += reward
        state = next_state
        if terminated or truncated:
            eval_rewards.append(episode_sum)
            break

avg_eval_reward = np.mean(eval_rewards)
print(f"Récompense moyenne sur {eval_episodes} épisodes d'évaluation: {avg_eval_reward:.2f}")
if avg_eval_reward >= 200:
    print(">>> Performance cible atteinte! L'agent obtient en moyenne au-dessus de 200 ✅")
else:
    print(">>> Performance cible NON atteinte. Réentraîner ou ajuster les hyperparamètres ⚠️")


Épisode 10/1000 - Score moyen (100 derniers): -176.6 - eps=0.951
Épisode 20/1000 - Score moyen (100 derniers): -170.4 - eps=0.905
Épisode 30/1000 - Score moyen (100 derniers): -164.4 - eps=0.860
Épisode 40/1000 - Score moyen (100 derniers): -157.8 - eps=0.818
Épisode 50/1000 - Score moyen (100 derniers): -150.5 - eps=0.778
Épisode 60/1000 - Score moyen (100 derniers): -150.8 - eps=0.740
Épisode 70/1000 - Score moyen (100 derniers): -155.2 - eps=0.704
Épisode 80/1000 - Score moyen (100 derniers): -152.7 - eps=0.670
Épisode 90/1000 - Score moyen (100 derniers): -148.2 - eps=0.637
Épisode 100/1000 - Score moyen (100 derniers): -144.3 - eps=0.606
Épisode 110/1000 - Score moyen (100 derniers): -139.2 - eps=0.576
Épisode 120/1000 - Score moyen (100 derniers): -135.4 - eps=0.548
Épisode 130/1000 - Score moyen (100 derniers): -128.1 - eps=0.521
Épisode 140/1000 - Score moyen (100 derniers): -124.9 - eps=0.496
Épisode 150/1000 - Score moyen (100 derniers): -119.8 - eps=0.471
Épisode 160/1000 - 

### Description

This environment is a classic rocket trajectory optimization problem. According to Pontryagin’s maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off.
There are two environment versions: discrete or continuous. The landing pad is always at coordinates (0,0). The coordinates are the first two numbers in the state vector. Landing outside of the landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt.

### Action Space

There are four discrete actions available:
- 0: do nothing
- 1: fire left orientation engine
- 2: fire main engine
- 3: fire right orientation engine

### Observation Space
The state is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

### Rewards
After every step a reward is granted. The total reward of an episode is the sum of the rewards for all the steps within that episode.
For each step, the reward:
- is increased/decreased the closer/further the lander is to the landing pad.
- is increased/decreased the slower/faster the lander is moving.
- is decreased the more the lander is tilted (angle not horizontal).
- is increased by 10 points for each leg that is in contact with the ground.
- is decreased by 0.03 points each frame a side engine is firing.
- is decreased by 0.3 points each frame the main engine is firing.

The episode receive an additional reward of -100 or +100 points for crashing or landing safely respectively.
An episode is considered a solution if it scores at least 200 points.

### Starting State
The lander starts at the top center of the viewport with a random initial force applied to its center of mass.

### Episode Termination
The episode finishes if:
- the lander crashes (the lander body gets in contact with the moon);
- the lander gets outside of the viewport (x coordinate is greater than 1);
- the lander is not awake. From the Box2D docs, a body which is not awake is a body which doesn’t move and doesn’t collide with any other body.

### Before submit
Test that your agent has the right attributes

In [22]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Pour gérer correctement l'absence potentielle de gymnasium sur la plateforme
try:
    import gymnasium as gym
except ImportError:
    print("gymnasium n'est pas disponible.")
    gym = None

###################################################################
# Réseau de neurones QNetwork pour approximer Q(s,a)
###################################################################
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        # Réseau fully-connected avec 2 couches cachées (128 neurones chacune)
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc_out = nn.Linear(128, action_dim)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc_out(x)


###################################################################
# Buffer d'expérience (ReplayBuffer)
###################################################################
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def add(self, state, action, reward, next_state, done):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = (
            np.array(state, copy=True),
            action,
            reward,
            np.array(next_state, copy=True),
            done
        )
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        indices = np.random.choice(len(self.memory), batch_size, replace=False)
        states, actions, rewards, next_states, dones = zip(*(self.memory[i] for i in indices))
        return (
            np.array(states),
            actions,
            rewards,
            np.array(next_states),
            dones
        )

    def __len__(self):
        return len(self.memory)


###################################################################
# Classe Agent (unique fichier, sans dépendre d'autres ressources)
###################################################################
class Agent:
    def __init__(self, env, player_name=None):
        """
        - env : environnement Gymnasium
        - player_name : nom du joueur (utile en multi-agent)
        """
        self.env = env

        # Gestion du cas multi-agent : la méthode action_space(...) renvoie un action_space
        # si c'est un environment multi-agent
        if self.env.action_space.__class__.__name__ == "method":
            # Pour un multi-agent, on sample directement
            self.action_space = self.env.action_space(self.env.agents[0])
            self.multi_agent = True
        else:
            # Environnement "classique" (ex: LunarLander-v2)
            self.action_space = self.env.action_space
            self.multi_agent = False

        # Dans le cas multi-agent, on ne s'embête pas : on agit aléatoirement
        # Dans le cas single-agent classique, on entraîne ici un DQN pour viser un score > 200
        if not self.multi_agent:
            self._train_dqn_for_lunarlander()

    def choose_action(self, observation, reward=0.0, terminated=False, truncated=False, info=None, action_mask=None):
        """
        Sélectionne et renvoie une action.
        - En multi-agent, on sample aléatoirement (car 'method').
        - En single-agent (LunarLander-v2), on exploite le réseau DQN entraîné.
        """
        if self.multi_agent:
            # multi-agent : on agit au hasard
            action = self.action_space.sample(action_mask)
        else:
            # single-agent : exploite le DQN entraîné
            state_t = torch.tensor(observation, dtype=torch.float32).unsqueeze(0)
            with torch.no_grad():
                q_values = self.q_network(state_t)
            action = int(torch.argmax(q_values, dim=1).item())
        return action

    ###################################################################
    # Entraînement DQN en local (inclut tout le code) pour LunarLander
    ###################################################################
    def _train_dqn_for_lunarlander(self):
        """
        Entraîne un agent DQN sur l'environnement self.env
        (supposé être un environnement Gymnasium LunarLander-v2).
        Stocke le réseau entraîné dans self.q_network.
        """
        # Vérifier qu'on a gymnasium
        if gym is None:
            print("Gymnasium introuvable, impossible d'entraîner le DQN.")
            self.q_network = None
            return

        # Hyperparamètres
        state_dim = self.env.observation_space.shape[0]
        action_dim = self.env.action_space.n
        max_episodes = 600        # nombre d'épisodes max (assez pour atteindre 200+)
        max_steps = 1000          # pas max par épisode
        gamma = 0.99
        lr = 5e-4
        batch_size = 64
        buffer_capacity = 100000
        update_rate = 4           # entraînement du réseau tous les 4 pas
        tau = 1e-3                # mise à jour douce du réseau cible
        target_score = 200        # objectif de performance
        min_episodes_for_avg = 100

        # Politique epsilon-greedy
        epsilon = 1.0
        epsilon_decay = 0.995
        epsilon_min = 0.01

        # Réseaux Q principal et cible
        self.q_network = QNetwork(state_dim, action_dim)
        self.target_network = QNetwork(state_dim, action_dim)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()

        optimizer = torch.optim.Adam(self.q_network.parameters(), lr=lr)
        replay_buffer = ReplayBuffer(buffer_capacity)

        scores_window = []
        best_avg_reward = -float("inf")

        episode = 0
        while episode < max_episodes:
            state, _ = self.env.reset()
            episode_reward = 0
            for t in range(max_steps):
                # Sélection action epsilon-greedy
                if random.random() < epsilon:
                    action = self.action_space.sample()
                else:
                    state_t = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
                    with torch.no_grad():
                        q_vals = self.q_network(state_t)
                    action = int(torch.argmax(q_vals, dim=1).item())

                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                replay_buffer.add(state, action, reward, next_state, done)

                state = next_state
                episode_reward += reward

                # Entraînement du réseau
                if t % update_rate == 0 and len(replay_buffer) >= batch_size:
                    self._dqn_train_step(replay_buffer, batch_size, optimizer, gamma, tau)

                if done:
                    break

            # Mise à jour epsilon
            epsilon = max(epsilon * epsilon_decay, epsilon_min)

            scores_window.append(episode_reward)
            if len(scores_window) > 100:
                scores_window.pop(0)
            episode += 1

            # Affichage
            avg_last_100 = np.mean(scores_window)
            if avg_last_100 > best_avg_reward:
                best_avg_reward = avg_last_100

            if episode % 25 == 0:
                print(f"[DQN training] Épisode {episode}/{max_episodes}, "
                      f"Reward: {episode_reward:.1f}, "
                      f"moyenne(100): {avg_last_100:.1f}, eps={epsilon:.3f}")

            # Arrêt si résolu
            if avg_last_100 >= target_score and episode >= min_episodes_for_avg:
                print(f"[DQN training] Environnement résolu en {episode} épisodes, moyenne(100)={avg_last_100:.1f}!")
                break

        print(f"[DQN training] Meilleure moyenne(100) atteinte = {best_avg_reward:.1f}")
        print("[DQN training] Fin de l'entraînement.")

    def _dqn_train_step(self, replay_buffer, batch_size, optimizer, gamma, tau):
        """
        Un pas d'entraînement du DQN : on échantillonne un mini-lot
        et on met à jour self.q_network et self.target_network.
        """
        states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

        # Conversion en tenseurs PyTorch
        states_t      = torch.tensor(states, dtype=torch.float32)
        actions_t     = torch.tensor(actions, dtype=torch.int64).unsqueeze(1)
        rewards_t     = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)
        next_states_t = torch.tensor(next_states, dtype=torch.float32)
        dones_t       = torch.tensor(dones, dtype=torch.float32).unsqueeze(1)

        # Q-cible via le réseau-cible
        with torch.no_grad():
            q_next = self.target_network(next_states_t).max(dim=1, keepdim=True)[0]
            q_target = rewards_t + gamma * q_next * (1 - dones_t)

        # Q-présent via le réseau principal
        q_current = self.q_network(states_t).gather(1, actions_t)

        # Perte MSE
        loss = F.mse_loss(q_current, q_target)

        # Rétropropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Mise à jour douce du réseau cible
        for target_param, local_param in zip(self.target_network.parameters(), self.q_network.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
